# Forward y Back propagation en Deep Learning
Implementaremos paso a paso tanto la propagación hacia adelante como hacia atrás.

En este notebook implementamos:

1. Forward Propagation:
   - Calcula las activaciones capa por capa
   - Usa ReLU para capas ocultas y Sigmoid para la capa de salida
   - Almacena valores intermedios en cache para backward propagation

2. Función de Costo:
   - Implementa binary cross-entropy
   - Incluye epsilon para evitar problemas numéricos con log(0)

3. Backward Propagation:
   - Calcula gradientes para cada capa
   - Implementa las fórmulas de la regla de la cadena
   - Calcula gradientes para pesos (W) y sesgos (b)

4. Actualización de Parámetros:
   - Implementa el descenso del gradiente
   - Actualiza W y b usando los gradientes calculados

También incluimos código de prueba para verificar las dimensiones y el funcionamiento básico.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from src.ft_functions import relu, relu_derivative, sigmoid, sigmoid_derivative

def forward_propagation(X, parameters):
    """
    Implementa la propagación hacia adelante
    
    Args:
        X (numpy.ndarray): Datos de entrada (n_features, n_samples)
        parameters (dict): Diccionario con los parámetros W1, b1, W2, b2, W3, b3
    
    Returns:
        dict: Diccionario con las activaciones y valores Z de cada capa
        tuple: Predicción final (Y_hat)
    """
    cache = {}
    
    # Primera capa oculta
    Z1 = np.dot(parameters['W1'], X) + parameters['b1']
    A1 = relu(Z1)
    cache['Z1'] = Z1
    cache['A1'] = A1
    
    # Segunda capa oculta
    Z2 = np.dot(parameters['W2'], A1) + parameters['b2']
    A2 = relu(Z2)
    cache['Z2'] = Z2
    cache['A2'] = A2
    
    # Capa de salida
    Z3 = np.dot(parameters['W3'], A2) + parameters['b3']
    A3 = sigmoid(Z3)  # Usamos sigmoid para la salida binaria
    cache['Z3'] = Z3
    cache['A3'] = A3
    
    return cache, A3

def compute_cost(A3, Y):
    """
    Calcula el costo usando binary cross-entropy
    
    Args:
        A3 (numpy.ndarray): Salida de la red (predicciones)
        Y (numpy.ndarray): Valores reales
    
    Returns:
        float: Costo calculado
    """
    m = Y.shape[1]
    
    # Añadimos epsilon para evitar log(0)
    epsilon = 1e-15
    A3 = np.clip(A3, epsilon, 1 - epsilon)
    
    cost = -(1/m) * np.sum(Y * np.log(A3) + (1 - Y) * np.log(1 - A3))
    
    return float(cost)

def backward_propagation(X, Y, parameters, cache):
    """
    Implementa la propagación hacia atrás
    
    Args:
        X (numpy.ndarray): Datos de entrada
        Y (numpy.ndarray): Valores reales
        parameters (dict): Parámetros de la red
        cache (dict): Valores almacenados del forward propagation
    
    Returns:
        dict: Gradientes para cada parámetro
    """
    m = Y.shape[1]
    gradients = {}
    
    # Gradientes de la capa de salida
    dZ3 = cache['A3'] - Y
    gradients['dW3'] = (1/m) * np.dot(dZ3, cache['A2'].T)
    gradients['db3'] = (1/m) * np.sum(dZ3, axis=1, keepdims=True)
    
    # Gradientes de la segunda capa oculta
    dA2 = np.dot(parameters['W3'].T, dZ3)
    dZ2 = dA2 * relu_derivative(cache['Z2'])
    gradients['dW2'] = (1/m) * np.dot(dZ2, cache['A1'].T)
    gradients['db2'] = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    
    # Gradientes de la primera capa oculta
    dA1 = np.dot(parameters['W2'].T, dZ2)
    dZ1 = dA1 * relu_derivative(cache['Z1'])
    gradients['dW1'] = (1/m) * np.dot(dZ1, X.T)
    gradients['db1'] = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    return gradients

def update_parameters(parameters, gradients, learning_rate):
    """
    Actualiza los parámetros usando descenso del gradiente
    
    Args:
        parameters (dict): Parámetros actuales
        gradients (dict): Gradientes calculados
        learning_rate (float): Tasa de aprendizaje
    
    Returns:
        dict: Parámetros actualizados
    """
    L = len(parameters) // 2  # Número de capas
    
    for l in range(1, L + 1):
        parameters[f'W{l}'] -= learning_rate * gradients[f'dW{l}']
        parameters[f'b{l}'] -= learning_rate * gradients[f'db{l}']
    
    return parameters

# Ejemplo de uso con datos pequeños para verificar dimensiones
np.random.seed(42)
X_sample = np.random.randn(30, 5)  # 5 muestras con 30 características
Y_sample = np.random.randint(0, 2, (1, 5))  # 5 etiquetas binarias

# Inicializar parámetros para el ejemplo
layer_dims = [30, 16, 8, 1]
parameters = {
    'W1': np.random.randn(16, 30) * np.sqrt(2./30),
    'b1': np.zeros((16, 1)),
    'W2': np.random.randn(8, 16) * np.sqrt(2./16),
    'b2': np.zeros((8, 1)),
    'W3': np.random.randn(1, 8) * np.sqrt(2./8),
    'b3': np.zeros((1, 1))
}

# Probar forward propagation
cache, A3 = forward_propagation(X_sample, parameters)

# Calcular costo
cost = compute_cost(A3, Y_sample)
print(f"Costo inicial: {cost}")

# Probar backward propagation
gradients = backward_propagation(X_sample, Y_sample, parameters, cache)

# Actualizar parámetros
learning_rate = 0.01
parameters = update_parameters(parameters, gradients, learning_rate)

# Verificar las dimensiones
print("\nDimensiones de las matrices:")
print(f"X: {X_sample.shape}")
print(f"Y: {Y_sample.shape}")
for key, value in parameters.items():
    print(f"{key}: {value.shape}")
for key, value in gradients.items():
    print(f"{key}: {value.shape}")

ModuleNotFoundError: No module named 'src'